# Foxtrot Man-Bikini Cafe

## Alpha: 1. Backaground:

Portland Oregon is located on the west coast of the United States. Oregon, Portland is right on the border between Washington and Oregon. It's the largest city by population in Oregon with different industries thriving. A client came to me asking if I could gather data on where to put her man-bikini coffee cafe called, "Foxtrot Zulu Bikini Cafe". It's an unique cafe where men are dressed in man-bikinis and serve coffee and other food items to customers. She picked Portland, Oregon because there's more of a population of females than males. So to her advantage she can advertise her coffee with men in sexy bathing suits serving coffee to customers. Target market of age demographics is between 20-35.

## Bravo: 2. Issue:

Oregon in general has an issue in the coffee market because there's an over supply of coffee shops with little demand of coffee. The issue with Portland is that there's too many cafes. Portland and the surrounding areas has the highest population in the state so it's ideal for someone who wants to open a cafe in Oregon. The data I will be gathering is by population, neighborhood, latitude, and longitude. 

## Charlie: 3. Data and how it will solve the issue:

For this project I will be web scraping by using beautifulSoup package to web scrape data from world-population-review of Oregon. I will also download data from portlandoregon.com that will provide neighborhoods, population of Portland, and how many females vs males in Portland. I will be using foursquare to populate all the cafes in different neihgborhoods. Finally I will then use the data I've gathered from neighborghood and the population and show her where to strategically place her business. I will do so by showing neighborhoods with the highest population with the least amount of cafes in X area. 

Oregon Population: 
http://worldpopulationreview.com/states/oregon-population/

Portland Population and neighborhoods:
https://www.portlandoregon.gov/civic/article/376383

Foursquare cafes in Portland Oregon:
https://foursquare.com/explore?mode=url&near=Portland%2C%20OR%2C%20United%20States&nearGeoId=72057594043674481&q=cafe

## Delta: 4. Downloading libraries that are needed:

In [1]:
try:
    print('Downloading needed packages...')
    !conda install -c conda-forge beautifulsoup4 --yes # web scraping
    !conda install -c conda-forge lxml --yes # parser for lxml web scraping
    !conda install -c conda-forge geocoder --yes # geocoder
    !conda install -c conda-forge folium --yes # for folium maps
    !conda install -c conda-forge xlrd --yes # excel
    print('Success: All lib downloaded')
except:
    print('Error')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    beautifulsoup4-4.6.3       |           py35_0         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following packages will be UPDATED:

    beautifulsoup4:  4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0        conda-forge
    ca-certificates: 2019.1.23-0          --> 2019.3.9-hecc5488_0 conda-forge
    certifi:         2018.8.24-py35_1     --> 2018.8.24-py35_1001 

## Echo: 4. Importing all the libraries needed:

In [2]:
try: 
    print('Importing libraries...')
    import numpy as np # library to handle data in a vectorized manner

    import pandas as pd # library for data analsysis
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    import types
    from botocore.client import Config
    import ibm_boto3
    
    import xlrd # excel

    import json # library to handle JSON files

    #!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

    import requests # library to handle requests
    from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

    # Matplotlib and associated plotting modules
    import matplotlib.cm as cm
    import matplotlib.colors as colors

    # import k-means from clustering stage
    from sklearn.cluster import KMeans

    #!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
    import folium # map rendering library

    print('Libraries imported.')
except: 
    print('Error: There was an error while importing libraries.')

Importing libraries...
Libraries imported.


## Foxtrot: 5. Importing excel file:

In [3]:
!wget -O portland_census.excel https://www.portlandoregon.gov/civic/article/376383 

--2019-06-08 10:43:27--  https://www.portlandoregon.gov/civic/article/376383
Resolving www.portlandoregon.gov (www.portlandoregon.gov)... 74.120.152.39
Connecting to www.portlandoregon.gov (www.portlandoregon.gov)|74.120.152.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184832 (180K) [application/vnd.ms-excel]
Saving to: ‘portland_census.excel’

100%[======================================>] 184,832      278KB/s   in 0.6s   

2019-06-08 10:43:29 (278 KB/s) - ‘portland_census.excel’ saved [184832/184832]



In [22]:
df_portland = pd.read_excel("portland_census.excel")
df_portland.head(20)

,Total Population,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Total races tallied,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Total population,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Households,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Total population.1,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Population under 18 years,Unnamed: 78,Population 65 years and over,Unnamed: 80,Unnamed: 81,Population in group quarters,Unnamed: 83,Unnamed: 84,Unnamed: 85,Housing units,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Population in occupied HUs,Unnamed: 97,Area,Unnamed: 99
NaN,NaN,Hispanic or latino origin by race,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Race alone or in combination with one or more...,NaN,NaN,NaN,NaN,NaN,Sex by age,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Household size by household type by presence o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Household type by relationship,NaN,NaN,NaN,NaN,NaN,NaN,Household type by relationship,NaN,NaN,NaN,NaN,Group quarters population by sex by age by GQ ...,NaN,NaN,NaN,NaN,Occupied housing units,NaN,NaN,Vacant housing units,NaN,NaN,NaN,NaN,NaN,By tenure,NaN,NaN,NaN
NaN,NaN,Not Hispanic or Latino,NaN,NaN,NaN,NaN,NaN,NaN,Hispanic or Latino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1-person household,NaN,2-or-more-person household,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In households,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tenure,NaN,NaN,Vacancy status,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Family households,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In family households,NaN,NaN,NaN,NaN,In nonfamily HHs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,Total,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some Other Race,Male Under 5 years,Male 5 to 9 years,Male 10 to 14 years,Male 15 to 17 years,Male 18 and 19 years,Male 20 to 24 years,Male 25 to 29 years,Male 30 to 34 years,Male 35 to 39 years,Male 40 to 44 years,Male 45 to 49 years,Male 50 to 54 years,Male 55 to 59 years,Male 60 to 64 years,Male 65 to 69 years,Male 70 to 74 years,Male 75 to 79 years,Male 80 to 84 years,Male 85 years and over,Female Under 5 years,Female 5 to 9 years,Female 10 to 14 years,Female 15 to 17 years,Female 18 and 19 years,Female 20 to 24 years,Female 25 to 29 years,Female 30 to 34 years,Female 35 to 39 years,Female 40 to 44 

## Golf: 6. Cleaning data

In [5]:
df_portland.shape

(136, 100)

In [6]:
df_portland = df_portland.iloc[3:,]

In [7]:
df_portland.head()

,Total Population,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Total races tallied,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Total population,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Households,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Total population.1,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Population under 18 years,Unnamed: 78,Population 65 years and over,Unnamed: 80,Unnamed: 81,Population in group quarters,Unnamed: 83,Unnamed: 84,Unnamed: 85,Housing units,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Population in occupied HUs,Unnamed: 97,Area,Unnamed: 99
NaN,Total,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some Other Race,Male Under 5 years,Male 5 to 9 years,Male 10 to 14 years,Male 15 to 17 years,Male 18 and 19 years,Male 20 to 24 years,Male 25 to 29 years,Male 30 to 34 years,Male 35 to 39 years,Male 40 to 44 years,Male 45 to 49 years,Male 50 to 54 years,Male 55 to 59 years,Male 60 to 64 years,Male 65 to 69 years,Male 70 to 74 years,Male 75 to 79 years,Male 80 to 84 years,Male 85 years and over,Female Under 5 years,Female 5 to 9 years,Female 10 to 14 years,Female 15 to 17 years,Female 18 and 19 years,Female 20 to 24 years,Female 25 to 29 years,Female 30 to 34 years,Female 35 to 39 years,Female 40 to 44 years,Female 45 to 49 years,Female 50 to 54 years,Female 55 to 59 years,Female 60 to 64 years,Female 65 to 69 years,Female 70 to 74 years,Female 75 to 79 years,Female 80 to 84 years,Female 85 years and over,Male householder,Female householder,Husband-wife family,"Husband-wife family, with own children under 1...","Male householder, no wife present","Male householder, no wife present, with own ch...","Female householder, no husband present","Female householder, no husband present, with o...",Nonfamily households,Households with one or more people under 18 years,Households with one or more people 65 years an...,Householder,Spouse,Child (Biological-Adopted or Stepchild),Relative other than Child and Spouse,Nonrelatives,Householder,Nonrelatives,In households,Own child,In households,Male householder living alone,Female householder living alone,Male institutionalized population,Male noninstitutionalized population,Female institutionalized population,Female noninstitutionalized population,Total,Owned with a mortgage or a loan,Owned free and clear,Renter occupied,For rent,For sale only,"Rented or sold, not occupied","For seasonal, recreational, or occasional use",For migrant workers,Other vacant,Population in owner occupied housing units,Population in renter occupied housing units,"Land area in square meters (divide by 2,589,98...","Water area in square meters (divide by 2,589,9..."
NEIGHBORHOOD OR PART,P0010001,P0050003,P0050004,P0050005,P0050006,P0050007,P0050008,P0050009,P0050011,P0050012,P0050013,P0050014,P0050015,P0050016,P0050017,P007000310,P007000411,P007000512,P007000613,P007000714,P007000815,P0

In [8]:
df_portland.shape

(133, 100)

In [9]:
df_portland = df_portland.drop(["Unnamed: 1", "Unnamed: 2", "Unnamed: 3", "Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12", "Unnamed: 13", "Unnamed: 14", "Total races tallied", "Unnamed: 16", "Unnamed: 17", "Unnamed: 18", "Unnamed: 19", "Unnamed: 20", "Unnamed: 22", "Unnamed: 23", "Unnamed: 32", "Unnamed: 33", "Unnamed: 34", "Unnamed: 35", "Unnamed: 36", "Unnamed: 37", "Unnamed: 38", "Unnamed: 39", "Unnamed: 40", "Unnamed: 41", "Unnamed: 42", "Unnamed: 43", "Unnamed: 53", "Unnamed: 54", "Unnamed: 55", "Unnamed: 56", "Unnamed: 57", "Unnamed: 58" "Unnamed: 60", "Unnamed: 61", "Unnamed: 62", "Unnamed: 63", "Unnamed: 64", "Unnamed: 65", "Unnamed: 66", "Unnamed: 67", "Unnamed: 68", "Unnamed: 69", "Unnamed: 71", "Unnamed: 72", "Unnamed: 73", "Unnamed: 74", "Unnamed: 75", "Unnamed: 75", "Unnamed: 76", "Population under 18 years", "Unnamed: 78", "Population 65 years and over", "Unnamed: 80", "Unnamed: 81", "Unnamed: 83", "Unnamed: 84", "Unnamed: 85", "Housing units", "Unnamed: 87", "Unnamed: 88", "Unnamed: 89", "Unnamed: 90", "Unnamed: 91", "Unnamed: 92", "Unnamed: 93", "Unnamed: 94", "Unnamed: 95", "Population in occupied HUs", "Unnamed: 97", "Area", "Unnamed: 99"], axis=1)

In [10]:
df_portland.head(20)

,Total Population,Total population,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Households,Total population.1,Population in group quarters
NaN,Total,Male Under 5 years,Male 5 to 9 years,Male 10 to 14 years,Male 15 to 17 years,Male 18 and 19 years,Male 20 to 24 years,Male 25 to 29 years,Male 30 to 34 years,Male 35 to 39 years,Male 40 to 44 years,Male 45 to 49 years,Male 50 to 54 years,Male 55 to 59 years,Male 60 to 64 years,Male 65 to 69 years,Male 70 to 74 years,Male 75 to 79 years,Male 80 to 84 years,Male 85 years and over,Female Under 5 years,Female 5 to 9 years,Female 10 to 14 years,Female 15 to 17 years,Female 18 and 19 years,Female 20 to 24 years,Female 25 to 29 years,Female 30 to 34 years,Female 35 to 39 years,Female 40 to 44 years,Female 45 to 49 years,Female 50 to 54 years,Female 55 to 59 years,Female 60 to 64 years,Female 65 to 69 years,Female 70 to 74 years,Female 75 to 79 years,Female 80 to 84 years,Female 85 years and over,Male householder,Householder,Male institutionalized population
NEIGHBORHOOD OR PART,P0010001,P0120003,P0120004,P0120005,P0120006,P0120007,P012000810,P0120011,P0120012,P0120013,P0120014,P0120015,P0120016,P0120017,P012001819,P012002021,P0120022,P0120023,P0120024,P0120025,P0120027,P0120028,P0120029,P0120030,P0120031,P012003234,P0120035,P0120036,P0120037,P0120038,P0120039,P0120040,P0120041,P012004243,P012004445,P0120046,P0120047,P0120048,P0120049,P0190003,P0290004,P043000424
ALAMEDA (PART),4200,175,173,142,78,17,57,65,71,185,214,168,144,186,173,80,50,28,20,13,147,145,113,79,31,58,45,129,202,210,156,187,220,181,104,63,34,27,30,88,1191,0
ALAMEDA/BEAUMONT-WILSHIRE,161,7,7,5,5,1,0,1,4,7,7,7,8,8,7,1,1,0,0,0,7,7,7,1,0,2,3,7,8,8,11,4,8,7,5,0,0,0,0,4,42,0
ALAMEDA/IRVINGTON,853,35,34,23,13,7,10,10,19,24,33,38,33,46,32,25,9,6,5,4,25,31,31,16,6,12,7,13,42,41,33,43,61,38,13,18,7,4,6,21,244,0
ARBOR LODGE,6153,173,166,134,62,32,160,289,349,370,276,246,214,197,157,98,63,40,22,28,180,140,104,70,45,168,318,368,369,236,171,219,200,179,107,66,38,41,58,383,1408,0
ARDENWALD-JOHNSON CREEK (PART),4480,132,133,143,87,45,106,149,183,189,161,156,166,170,125,80,45,33,30,23,144,115,116,81,42,112,156,201,163,160,179,196,176,155,97,65,54,56,56,259,1124,0
ARDENWALD-JOHNSON CREEK/WOODSTOCK,181,5,4,7,6,2,2,8,5,9,8,7,7,10,6,2,0,3,1,1,5,1,7,2,0,5,10,5,9,6,8,6,8,8,4,0,1,1,2,7,48,0
ARGAY (PART),5990,205,201,207,131,65,201,207,197,154,186,194,204,205,145,109,101,72,71,62,179,175,164,132,77,200,190,193,165,162,193,225,203,194,163,110,108,96,144,248,1486,5
ARGAY/WILKES,16,0,0,0,0,2,1,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,3,0,1,1,0,0,0,0,0,0,5,0


In [20]:
df_portland.rename(
    columns={
        "Total Population": "Total",
        "Total population": "Male Under 5 years",
        "Unnamed: 22": "Male 5 to 9 years",
        "Unnamed: 23": "to 9 years Male 10 to 14 years",
        "Unnamed: 24": "14 years Male 15 to 17 years",
        "Unnamed: 25": "Male 18 and 19 years",
        "Unnamed: 26": "Male 20 to 24 years",
        "Unnamed: 27": "Male 25 to 29 years",
        "Unnamed: 28": "Male 30 to 34 years",
        "Unnamed: 29": "Male 35 to 39 years",
        "Unnamed: 30": "Male 40 to 44 years",
        "Unnamed: 31": "Male 45 to 49 years",
        "Unnamed: 32": "Male 50 to 54 years",
        "Unnamed: 33": "Male 55 to 59 years",
        "Unnamed: 34": "Male 60 to 64 years",
        "Unnamed: 35": "Male 65 to 69 years",
        "Unnamed: 36": "Male 70 to 74 years",
    },
    inplace=True
)

In [21]:
df_portland.head()

,Total,Total population,Male 5 to 9 years,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Households,Total population.1,Population in group quarters
NaN,Total,Male Under 5 years,Male 5 to 9 years,Male 10 to 14 years,Male 15 to 17 years,Male 18 and 19 years,Male 20 to 24 years,Male 25 to 29 years,Male 30 to 34 years,Male 35 to 39 years,Male 40 to 44 years,Male 45 to 49 years,Male 50 to 54 years,Male 55 to 59 years,Male 60 to 64 years,Male 65 to 69 years,Male 70 to 74 years,Male 75 to 79 years,Male 80 to 84 years,Male 85 years and over,Female Under 5 years,Female 5 to 9 years,Female 10 to 14 years,Female 15 to 17 years,Female 18 and 19 years,Female 20 to 24 years,Female 25 to 29 years,Female 30 to 34 years,Female 35 to 39 years,Female 40 to 44 years,Female 45 to 49 years,Female 50 to 54 years,Female 55 to 59 years,Female 60 to 64 years,Female 65 to 69 years,Female 70 to 74 years,Female 75 to 79 years,Female 80 to 84 years,Female 85 years and over,Male householder,Householder,Male institutionalized population
NEIGHBORHOOD OR PART,P0010001,P0120003,P0120004,P0120005,P0120006,P0120007,P012000810,P0120011,P0120012,P0120013,P0120014,P0120015,P0120016,P0120017,P012001819,P012002021,P0120022,P0120023,P0120024,P0120025,P0120027,P0120028,P0120029,P0120030,P0120031,P012003234,P0120035,P0120036,P0120037,P0120038,P0120039,P0120040,P0120041,P012004243,P012004445,P0120046,P0120047,P0120048,P0120049,P0190003,P0290004,P043000424
ALAMEDA (PART),4200,175,173,142,78,17,57,65,71,185,214,168,144,186,173,80,50,28,20,13,147,145,113,79,31,58,45,129,202,210,156,187,220,181,104,63,34,27,30,88,1191,0
ALAMEDA/BEAUMONT-WILSHIRE,161,7,7,5,5,1,0,1,4,7,7,7,8,8,7,1,1,0,0,0,7,7,7,1,0,2,3,7,8,8,11,4,8,7,5,0,0,0,0,4,42,0
ALAMEDA/IRVINGTON,853,35,34,23,13,7,10,10,19,24,33,38,33,46,32,25,9,6,5,4,25,31,31,16,6,12,7,13,42,41,33,43,61,38,13,18,7,4,6,21,244,0
